In [1]:
import pandas as pd
import numpy as np
from numpy import dstack
import logging
import itertools
import os
import gensim
import nltk
import re

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.utils import to_categorical
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import text, sequence

from nltk.corpus import stopwords



tf.test.gpu_device_name()

Using TensorFlow backend.


''

In [2]:
Mobile = pd.read_csv('Mobile_shuffled.csv')

In [3]:
train_size = int(len(Mobile) * 0.8) 
train_title = Mobile['title'][:train_size] #Takes 70% of the training data to train
train_Category = Mobile['Category'][:train_size] #Takes 70% of the training data to train

eval_title = Mobile['title'][train_size:] #Takes remaining 30% of the training data to test
eval_Category = Mobile['Category'][train_size:] #Takes remaining 30% of the training data to test

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_title) # only fit on train

x_train = tokenize.texts_to_matrix(train_title)
x_eval = tokenize.texts_to_matrix(eval_title)

encoder = LabelEncoder()
encoder.fit(train_Category)
y_train = train_Category
y_eval = eval_Category
y_train = encoder.transform(train_Category)
y_eval = encoder.transform(eval_Category)


num_classes = np.max(y_train) - np.min(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_eval = utils.to_categorical(y_eval, num_classes)


In [4]:
from keras.models import load_model

def load_all_models(n_models):
    all_models = list()
    for i in range(3):
        filename = 'mobile_models/model_' + str(i + 1) + '.h5'
        model = load_model(filename)                   
        all_models.append(model)                        
        print('>loaded %s' % filename)
    return all_models


In [5]:
members = load_all_models(3)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
>loaded mobile_models/model_1.h5
>loaded mobile_models/model_2.h5
>loaded mobile_models/model_3.h5


In [6]:
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:                         # make prediction for each sub-model
        yhat = model.predict(inputX, verbose=0)   # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))       # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX




def fit_stacked_model(members, inputX, inputy):     # fit models based on the outputs from the ensemble members
    stackedX = stacked_dataset(members, inputX)     # create dataset using ensemble
    model = LogisticRegression()                    # fit standalone model
    model.fit(stackedX, np.argmax(inputy, axis=1))
    return model


model = fit_stacked_model(members, x_eval,y_eval)

/Users/yeeeeehan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yeeeeehan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [7]:
def stacked_prediction(members, model, inputX):
    stackedX = stacked_dataset(members, inputX)
    yhat = model.predict(stackedX)
    return yhat

yhat = stacked_prediction(members, model, x_eval)             # evaluate model on test set
y_val_values = ([np.argmax(y, axis=None, out=None) for y in y_eval])
acc = accuracy_score(y_val_values, yhat)
print('Stacked Test Accuracy: %.3f' % acc)


Stacked Test Accuracy: 0.817


In [9]:
testset = pd.read_csv('test.csv')
mobile_test = testset.iloc[131985:]
test_title = mobile_test['title']
x_test = tokenize.texts_to_matrix(test_title)
score = stacked_prediction(members, model, x_test)
mobile_predict = pd.DataFrame(data=score)
mobile_predict = beauty_predict.idxmax(axis=1)
mobile_predict.to_csv('mobile_predict.csv')

NameError: name 'beauty_predict' is not defined